# Energy Use Analysis

# Introduction
This is data from my energy company, ComEd from the past year. 10_22_2022 to 10_22_2023
## Goals
* become familiar with the columns in the dataset

## Source
Data from the [My Green Button] (https://secure.comed.com/MyAccount/MyBillUsage/pages/secure/GreenButtonConnectDownloadMyData.aspx) webpage on the ComEd website.

In [25]:
import pandas as pd
import numpy as np
energy_df = pd.read_csv(filepath_or_buffer="energy_use_10_22_22_to_10_22_23.csv", header=4)

In [26]:
print(energy_df.columns)
energy_df.head()

Index(['TYPE', 'DATE', 'START TIME', 'END TIME', 'USAGE', 'UNITS', 'COST',
       'NOTES'],
      dtype='object')


,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST,NOTES
0,Electric usage,2022-10-22,00:00,00:29,0.11,kWh,$0.01,NaN
1,Electric usage,2022-10-22,00:30,00:59,0.13,kWh,$0.02,NaN
2,Electric usage,2022-10-22,01:00,01:29,0.09,kWh,$0.01,NaN
3,Electric usage,2022-10-22,01:30,01:59,0.20,kWh,$0.02,NaN
4,Electric usage,2022-10-22,02:00,02:29,0.10,kWh,$0.01,NaN


# Column / header info
* **col0**: description0
* **col1**: description1

# Initial Observations
* Observation0
* Observation1

In [ ]:
df.info()

In [ ]:
df.describe

# Cleaning

# Ideas for Improvement
* 
* 